# 05. Azure Form Recognizer results to Pandas & Excel

<img src="https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/media/tables-example.jpg#lightbox">

> https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import json
import time
import pandas as pd
from requests import get, post

In [3]:
def extract_value(value):
    """
    Helper Method to Extract Cell Value from Response
    """
    if value['type'] == 'number':
        return value['text']
    elif value['type'] == 'string':
        return value['valueString']
    elif value['type'] == 'date':
        return value['valueDate']            
    elif value['type'] == 'time':
        return value['valueTime']
    elif value['type'] == 'phoneNumber':
        return value['valuePhoneNumber']
    elif value['type'] == 'object':
        objectKeys = value['valueObject'].keys();
        item_info = "" 
        for ok in objectKeys:
            item_info += ok + ":" + extract_value(value['valueObject'][ok]) + " "
        return item_info
    elif value['type'] == 'array':
        itemInfo = ""
        for item in value["valueArray"]:
            itemInfo += extract_value(item) + "; "
        return itemInfo[:-3] # ; 
    else:
        print("Skipping Unsupported Type")

def recognizer2DF(post_url, apim_key, headers, data_bytes, confidence_threshold = 0, query_interval=5):
    """
    Submits Table or Form to recognizer asyncronously and processes the response
    queryInterval amount of time to wait between checking whether a job is done
    Optional confidence_threshold to deterimine whether to process a extracted feild 
    """
    try:
        # Submit Async Table Job to Form Recognizer Endpoint 
        resp = post(url = post_url, data = data_bytes, headers = headers)
        if resp.status_code == 202:
            # Query Submit Table Job
            get_url = resp.headers["operation-location"]
            resp = get(url = resp.headers["operation-location"], headers = {"Ocp-Apim-Subscription-Key": apim_key})
            resp_json = json.loads(resp.text)
            while resp_json["status"] == "running":
                resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
                resp_json = json.loads(resp.text)
                time.sleep(query_interval)
            if resp_json["status"] == "succeeded":
                # Process Documents 
                docResults = resp_json['analyzeResult']['documentResults']
                docs = []
                for doc in docResults:
                    fields = doc['fields']
                    docs.append({key:extract_value(fields[key]) for key in fields.keys() \
                                 if 'confidence' in fields[key] and fields[key]['confidence'] > confidence_threshold}) 
                return pd.DataFrame(docs)
            elif resp_json["status"] == "failed":
                print("Layout analyze failed:\n%s" % resp_json)
        else:
            print("POST analyze failed:\n%s" % resp.text)     
    except Exception as e:
        print("Code Failed analyze failed:\n%s" % str(e))

In [4]:
apim_key = "yourkey"
endpoint =  "yourendpoint/formrecognizer/v2.1-preview.1/prebuilt/receipt/analyze"

In [5]:
source = "receipt.jpg"

In [9]:
headers = {
    # Request headers
    'Content-Type': 'image/jpeg',
    'Ocp-Apim-Subscription-Key': apim_key,
}

with open(source, "rb") as f:
    data_bytes = f.read()

In [10]:
df = recognizer2DF(endpoint, apim_key, headers, data_bytes)

In [11]:
df.dtypes

MerchantAddress        object
MerchantName           object
MerchantPhoneNumber    object
ReceiptType            object
Subtotal               object
Tax                    object
Tip                    object
Total                  object
TransactionDate        object
TransactionTime        object
dtype: object

In [12]:
df.head()

,MerchantAddress,MerchantName,MerchantPhoneNumber,ReceiptType,Subtotal,Tax,Tip,Total,TransactionDate,TransactionTime
0,"123 Main Street Redmond, WA 98052",Contoso Contoso,+19876543210,Itemized,11.70,1.17,1.63,$14.50,2020-06-10,13:59:00


In [13]:
df.to_csv("extraction.csv")
df.to_excel("extraction.xlsx")
!ls extraction.* -l

-rwxrwxrwx 1 root root  236 Jun  2 14:50 extraction.csv
-rwxrwxrwx 1 root root 5646 Jun  2 14:50 extraction.xlsx
